In [1]:
import os
import pandas as pd
import numpy as np
import re

from urllib import parse

import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

In [2]:
bigDict = {'/jobse/' : '채용/인재',
           '/magazine/' : '매거진',
           '/inews/' : '뉴스',
           'https://edu.lafent.com' : 'e-아카데미',
           '/mtrial/' : '작품/조경수/자재',
           '/photo/' : '포토갤러리',
           '/sns/' : '커뮤니티',
           '/help/' : '고객센터',
           '/myinfo/' : '마이페이지',
           '/search/' : '검색'}
    

def findBig(x, bigDict = bigDict):
    for key in bigDict.keys():
        if key in x:
            return bigDict[key]
    return '기타'  

In [3]:
# 기타(크롤링)을 제외한 로그 데이터 
innerOut_path = '../Data/Preprocessed/innerOut.csv'
innerOut_df_origin = pd.read_csv(innerOut_path)
innerOut_df_origin['Timestamp'] = pd.to_datetime(innerOut_df_origin['Timestamp'])
innerOut_df_origin.loc[:, 'URL'] = innerOut_df_origin.loc[:, 'URL'].str.lower()
innerOut_df_origin.loc[:, 'URL'] = innerOut_df_origin.loc[:, 'URL'].apply(lambda x : parse.unquote(x))
innerOut_df_origin.loc[:, 'URL'] = innerOut_df_origin.loc[:, 'URL'].str.replace('http://', '')
innerOut_df_origin.loc[:, 'URL'] = innerOut_df_origin.loc[:, 'URL'].str.replace('https://', '')
innerOut_df_origin.loc[:, 'URL'] = innerOut_df_origin.loc[:, 'URL'].str.replace('lafent.com/', '/')
innerOut_df_origin.loc[:, 'URL'] = innerOut_df_origin.loc[:, 'URL'].str.replace('/mbweb/', '/')
innerOut_df_origin.loc[:, 'bigPath'] = innerOut_df_origin.loc[:, 'URL'].progress_apply(lambda x : findBig(x)).values

innerOut_df_origin['mcd'] = innerOut_df_origin['URL'].str.extract(r"(mcd=[a-z]\d{2})")
innerOut_df_origin['rk'] = innerOut_df_origin['URL'].str.extract(r"(rk=[a-z])")
innerOut_df_origin['bt'] = innerOut_df_origin['URL'].str.extract(r"(bt=[a-z])")
innerOut_df_origin['bsp'] = innerOut_df_origin['URL'].str.extract(r"(bsp=[a-z0-9]{2})")
innerOut_df_origin[['mcd', 'rk', 'bt', 'bsp']] = innerOut_df_origin[['mcd', 'rk', 'bt', 'bsp']].fillna('NA')

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_21216\18505219.py:3: DtypeWarning: Columns (4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  innerOut_df_origin = pd.read_csv(innerOut_path)
C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_21216\18505219.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  innerOut_df_origin.loc[:, 'URL'] = innerOut_df_origin.loc[:, 'URL'].str.replace('lafent.com/', '/')
100%|██████████| 3119639/3119639 [00:04<00:00, 632110.70it/s]


In [4]:
innerOut_df_origin.head(2)

,Timestamp,IP,URL,Category,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,Referer Content Type,...,New Category,User Type,month,day,hour,bigPath,mcd,rk,bt,bsp
0,2022-08-01 14:30:55+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=24596,jobs,NaN,NaN,NaN,NaN,etc,etc,...,jobs,내부,8,1,14,채용/인재,mcd=c01,NA,NA,NA
1,2022-08-01 14:32:14+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=23934,jobs,NaN,NaN,NaN,NaN,etc,etc,...,jobs,내부,8,1,14,채용/인재,mcd=c01,NA,NA,NA


In [5]:
# 경로 데이터
path_path = '../Data/Raw/category.xlsx'
path_df_origin = pd.read_excel(path_path)
path_df_origin = path_df_origin.fillna(method = 'ffill')
path_df_origin.loc[(path_df_origin['path'].str[0] != '/') & ~(path_df_origin['path'].str.contains('http')), 'path'] = path_df_origin.loc[(path_df_origin['path'].str[0] != '/') & ~(path_df_origin['path'].str.contains('http')), 'path'].apply(lambda x : '/' + x)
path_df_origin['path'] = path_df_origin['path'].str.lower()
path_df_origin['mcd'] = path_df_origin['path'].str.extract(r"(mcd=[a-z]\d{2})")
path_df_origin['rk'] = path_df_origin['path'].str.extract(r"(rk=[a-z])")
path_df_origin['bt'] = path_df_origin['path'].str.extract(r"(bt=[a-z])")
path_df_origin['bsp'] = path_df_origin['path'].str.extract(r"(bsp=[a-z0-9]{2})")
path_df_origin['bigPath'] = path_df_origin['path'].progress_apply(lambda x : findBig(x))
path_df_origin[['mcd', 'rk', 'bt', 'bsp']] = path_df_origin[['mcd', 'rk', 'bt', 'bsp']].fillna('NA')

# path_df_origin.to_csv('../Data/Preprocessed/category.csv',
#                       index = False, 
#                       encoding = 'utf-8-sig', 
#                       escapechar = '\\')

100%|██████████| 744/744 [00:00<00:00, 372693.44it/s]


In [6]:
def makeCategory(x, path_df):
    try:
        if x['URL'] == '/':
            return '메인', '메인'

        else:
            path_df.loc[:, 'tmp'] = x['URL']
            result = path_df[path_df.apply(lambda x : x['path'] in x['tmp'], axis = 1)]

            if len(result) != 0:
                return result['middlePath'].values[0], result['smallPath'].values[0]
            
            else:
                result = path_df.loc[(path_df['bigPath'] == x['bigPath']) & (path_df['mcd'] == x['mcd']) & (path_df['rk'] == x['rk']) & (path_df['bt'] == x['bt']) & (path_df['bsp'] == x['bsp'])]
                return result['middlePath'].values[0], result['smallPath'].values[0]                   
    
    except:
        return 'error', 'error'


In [7]:
innerOut_df_origin[['middlePath', 'smallPath']] = innerOut_df_origin.progress_apply(lambda x: makeCategory(x, path_df_origin), axis = 1, result_type = 'expand')
innerOut_df_origin.loc[(innerOut_df_origin['bigPath'] == '기타') & (innerOut_df_origin['middlePath'] == '메인') & (innerOut_df_origin['smallPath'] == '메인'), ['middlePath', 'smallPath']] = ['기타', '기타']
# innerOut_df_origin.to_csv('../Data/Preprocessed/innerOutTaged.csv',
#                           encoding = 'utf-8-sig',
#                           escapechar = '\\',
#                           index = False)

100%|██████████| 3119639/3119639 [7:56:52<00:00, 109.03it/s]  


KeyError: 'pathBig'

In [9]:
# 추가 전처리 01
innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')
innerOut_df_origin['tmp'] = innerOut_df_origin['URL'].apply(lambda x : x.split('/')[1] if len(x.split('/')) > 1 else 'error')

innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'mcd'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'URL'].str.extract(r"(mcd=[a-z]\d{2})").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'rk'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'URL'].str.extract(r"(rk=[a-z])").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'bt'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'URL'].str.extract(r"(bt=[a-z])").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'bsp'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'URL'].str.extract(r"(bsp=[a-z0-9]{2})").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), ['mcd', 'rk', 'bt', 'bsp']] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), ['mcd', 'rk', 'bt', 'bsp']].fillna('NA').values

innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'mcd'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'URL'].str.extract(r"(mcd=[a-z]\d{2})").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'rk'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'URL'].str.extract(r"(rk=[a-z])").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'bt'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'URL'].str.extract(r"(bt=[a-z])").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'bsp'] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'URL'].str.extract(r"(bsp=[a-z0-9]{2})").values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), ['mcd', 'rk', 'bt', 'bsp']] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), ['mcd', 'rk', 'bt', 'bsp']].fillna('NA').values

innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), 'bigPath'] = '채용/인재'
innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), 'bigPath'] = '뉴스'

innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job'), ['middlePath', 'smallPath']] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'] == 'job')].progress_apply(lambda x : makeCategory(x, path_df_origin), axis = 1, result_type = 'expand').values
innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2'])), ['middlePath', 'smallPath']] = innerOut_df_origin.loc[(innerOut_df_origin['tmp'].isin(['news', 'news2']))].progress_apply(lambda x : makeCategory(x, path_df_origin), axis = 1, result_type = 'expand').values

# innerOut_df_origin.to_csv('../Data/Preprocessed/innerOutTaged.csv',
#                           encoding = 'utf-8-sig',
#                           escapechar = '\\',
#                           index = False)

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_21216\56883465.py:2: DtypeWarning: Columns (4,5,6,7,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')
100%|██████████| 275133/275133 [39:41<00:00, 115.55it/s]


In [10]:
# 추가 전처리 02
innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')
innerOut_df_origin.loc[innerOut_df_origin['URL'].str.contains('edu'), ['bigPath', 'middlePath', 'smallPath']] = ['e-아카데미', 'e-아카데미', 'e-아카데미']
innerOut_df_origin['news_id'] = innerOut_df_origin['URL'].str.extract('(news_id=\d*)')
innerOut_df_origin['idx'] = innerOut_df_origin['URL'].str.extract('(idx=\d*)')

tmp01 = innerOut_df_origin.loc[(innerOut_df_origin['middlePath'] == '인기통합정보'), 'bigPath'].values
tmp02 = innerOut_df_origin.loc[(innerOut_df_origin['middlePath'] == '인기통합정보'), 'smallPath'].values
innerOut_df_origin.loc[(innerOut_df_origin['middlePath'] == '인기통합정보'), 'bigPath'] = '인기통합정보'
innerOut_df_origin.loc[(innerOut_df_origin['bigPath'] == '인기통합정보'), 'middlePath'] = tmp02
innerOut_df_origin.loc[(innerOut_df_origin['bigPath'] == '인기통합정보'), 'smallPath'] = tmp01
# innerOut_df_origin.to_csv('../Data/Preprocessed/innerOutTaged.csv', encoding = 'utf-8-sig', escapechar = '\\', index = False)

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_21216\1906809498.py:2: DtypeWarning: Columns (4,5,6,7,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')


# 2023/10/22(일) 추가 전처리 진행

In [4]:
path_df_origin = pd.read_csv('../Data/Preprocessed/category.csv')
path_df_origin = path_df_origin.fillna('NA')

,bigPath,middlePath,smallPath,path,mcd,rk,bt,bsp
0,기타,메인,메인,/index.html,NaN,NaN,NaN,NaN
1,뉴스,뉴스홈,뉴스홈,/inews/index.html,NaN,NaN,NaN,NaN


In [10]:
path_df_origin.head(2)

,bigPath,middlePath,smallPath,path,mcd,rk,bt,bsp
0,기타,메인,메인,/index.html,NA,NA,NA,NA
1,뉴스,뉴스홈,뉴스홈,/inews/index.html,NA,NA,NA,NA


In [25]:
def makeCategory_version02(x, path_df):
    try:
        if x['URL'] == '/':
            return '메인', '메인'

        else:
            path_df.loc[:, 'tmp'] = x['URL']
            result = path_df[path_df.apply(lambda x : x['path'] in x['tmp'], axis = 1)]

            if len(result) != 0:
                return result['middlePath'].values[0], result['smallPath'].values[0]
            
            else:
                result = path_df.loc[(path_df['bigPath'] == x['bigPath']) & (path_df['mcd'] == x['mcd'])]
                return result['middlePath'].values[0], result['smallPath'].values[0]                   
    
    except:
        return 'error', 'error'


In [33]:
innerOut_path = '../Data/Preprocessed/innerOutTaged.csv'
innerOut_df_origin = pd.read_csv(innerOut_path)
innerOut_df_origin[['mcd', 'rk', 'bt', 'bsp']] = innerOut_df_origin[['mcd', 'rk', 'bt', 'bsp']].fillna('NA')
innerOut_df_origin.loc[innerOut_df_origin['middlePath'] == 'error', ['middlePath', 'smallPath']] = innerOut_df_origin.loc[innerOut_df_origin['middlePath'] == 'error'].progress_apply(lambda x : makeCategory_version02(x, path_df_origin), axis = 1, result_type = 'expand').values
innerOut_df_origin.loc[(innerOut_df_origin['URL'] == '/index.html') & (innerOut_df_origin['bigPath'] == '기타'), ['bigPath', 'middlePath', 'smallPath']] = ['메인', '메인', '메인']
# innerOut_df_origin.to_csv('../Data/Preprocessed/innerOutTaged.csv',
#                           encoding = 'utf-8-sig',
#                           escapechar = '\\',
#                           index = False)

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_13860\277475096.py:2: DtypeWarning: Columns (4,5,6,7,20,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  innerOut_df_origin = pd.read_csv(innerOut_path)
100%|██████████| 20445/20445 [04:34<00:00, 74.51it/s]
